In [1]:
import numpy as np
import scipy
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import ParameterVector
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp, Statevector, state_fidelity, Operator
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.primitives import Estimator
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from scipy.linalg import expm, cosm
import warnings

In [3]:
def get_hamiltonian(nqubits):
    J=1/np.sqrt(2)
    ZZ_tuples = [('ZZ', [i, i+1], J) for i in range(nqubits-1)]
    ZZ_tuples += [('ZZ', [nqubits-1, 0], J)]
    X_tuples = [("X", [i], J) for i in range(nqubits)]
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *X_tuples], num_qubits=nqubits)
    return hamiltonian.simplify()
    
def ansatz_hamiltonian(nqubits, depth, params):
    circuit = QuantumCircuit(nqubits)
    t = 0
    for i in range(nqubits):
        circuit.h(i)
    for idepth in range(depth):
        for i in range(nqubits):
            if i < (nqubits - 1):
                j = i + 1
            else:
                j = 0
            circuit.rzz(params[t], i, j)
            t += 1
        circuit.barrier()
        for i in range(nqubits):
            if i < (nqubits - 1):
                j = i + 1
            else:
                j = 0    
            circuit.rx(params[t], i)
            t += 1
    return circuit    
    
def ansatz_hea(nqubits, depth, params):
    circuit = QuantumCircuit(nqubits)
    t = 0
    for idepth in range(depth):
        for i in range(nqubits):
            circuit.ry(params[t], i)
            t += 1
        circuit.barrier()
        for i in range(nqubits-1):
            circuit.cx(i, i+1)
    return circuit    
   
def cost_function1(params, nqubits, depth, error_rate):
    hamiltonian =  get_hamiltonian(nqubits)
    circuit =  ansatz_hamiltonian(nqubits, depth,params)
    circuit = circuit.decompose()
    noise_model = NoiseModel()
    error = depolarizing_error(error_rate, 1)
    noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
    error1 = depolarizing_error(error_rate*10, 2)
    noise_model.add_all_qubit_quantum_error(error1,'cx')
    sim_d = AerSimulator(noise_model=noise_model)
    circuit.save_statevector() 
    if error_rate == 0:
        simulator = AerSimulator()
    else:
        simulator = sim_d
        circuit = transpile(circuit, sim_d)
        #noise_result = sim_d.run(circ_noise, shots=1).result()
    result = simulator.run(circuit).result()
    u=result.data(0)['statevector'].data
    expectation = (u.conj().dot(hamiltonian.to_matrix())).dot(u)
    return expectation.real
    
def cost_function2(nqubits, error_rate, params):
    hamiltonian =  get_hamiltonian(nqubits)
    circuit =  ansatz_hea(nqubits, depth,params)
    circuit = circuit.decompose()
    noise_model = NoiseModel()
    error = depolarizing_error(error_rate, 1)
    noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
    error1 = depolarizing_error(error_rate*10, 2)
    noise_model.add_all_qubit_quantum_error(error1,['cx'])
    sim_d = AerSimulator(noise_model=noise_model)
    circuit.save_statevector() 
    if error_rate == 0:
        simulator = AerSimulator()
    else:
        simulator = sim_d
        circuit = transpile(circuit, sim_d)
        #noise_result = sim_d.run(circ_noise, shots=1).result()
    result = simulator.run(circuit).result()
    u=result.data(0)['statevector'].data
    expectation = (u.conj().dot(hamiltonian.to_matrix())).dot(u)
    return expectation.real

def callback(intermediate_result):
    print(
        f"{intermediate_result.fun}"
        )
    #with open('intermediate_values_hva.txt', 'a') as file:
     #   file.write(f'Intermediate values: {intermediate_result.fun}\n')
    


In [5]:
np.random.seed(42)  # Example seed
nqubits = 4
J = 1/np.sqrt(2)
depth = 2
error_rate = 0
parameter = np.array(np.random.random(2*nqubits*depth))
#ansatz_hamiltonian(nqubits, depth, parameter).draw('mpl')
estimate_val_hva = minimize(cost_function1, parameter, args=(nqubits, depth, error_rate), method="BFGS", tol=1e-5, options={'disp': False}, callback=callback)

-1.1465461308365552
-2.602898241772969
-2.8480939891654384
-3.206002278353627
-3.3199939754754464
-3.438404344210988
-3.5373622592711627
-3.605472814784278
-3.6484027346773726
-3.6620229855816957
-3.670442800287261
-3.6765937081215974
-3.6807428442385475
-3.6865486627872173
-3.6930313640854613
-3.6947920036244293
-3.695177507255287
-3.6953272872091683
-3.695444871761792
-3.6954921867071837
-3.695513649551492
-3.695517966971842
-3.695518114519997
-3.695518129751825
-3.695518130034963


In [6]:
np.random.seed(42)  # Example seed
nqubits = 8
J = 1/np.sqrt(2)
depth = 2
error_rate = 0
parameter = np.array(np.random.random(2*nqubits*depth))
#ansatz_hamiltonian(nqubits, depth, parameter).draw('mpl')
estimate_val_hva = minimize(cost_function1, parameter, args=(nqubits, depth, error_rate), method="BFGS", tol=1e-5, options={'disp': False}, callback=callback)#, args=(nqubits, error_rate)

0.6637232518528577
-2.40833751534997
-3.376825013192576
-4.905611179564476
-6.339468350446939
-6.662111666376175
-6.697645374358425
-6.757550240624189
-6.841144239198982
-6.876494078547202
-6.882978749239546
-6.8937327561917225
-6.905295595482091
-6.908614481161706
-6.909772734261383
-6.9115779741636985
-6.9145867187368975
-6.919421820869848
-6.923994235183605
-6.930226088274259
-6.939068030151231
-6.9411285420260445
-6.943582647196257
-6.945164993365611
-6.947219853564121
-6.9502666493898255
-6.953056501903131
-6.9558250262688315
-6.9605480935505355
-6.966914296278535
-6.972036370831583
-6.978253905680795
-6.9812821821678925
-6.9858975531894885
-6.990173694665251
-6.994656476501433
-7.000036853225115
-7.002672306754281
-7.006420586791361
-7.009574907881002
-7.011718429606295
-7.013894957073203
-7.015518146984938
-7.016517539550372
-7.0173054833311435
-7.0179965660846
-7.018473678132986
-7.018746188778917
-7.018921421827924
-7.019081934893309
-7.019274932173822
-7.01935060622259
-7.019

In [74]:
hamiltonian = get_hamiltonian(4)
H_array = hamiltonian.to_matrix()
eval, _ = np.linalg.eigh(H_array)
# print(eval)
emin = eval[0]
emax = eval[-1]
print(emin)

-3.695518130045147


In [71]:
hamiltonian = get_hamiltonian(8)
H_array = hamiltonian.to_matrix()
eval, _ = np.linalg.eigh(H_array)
# print(eval)
emin = eval[0]
emax = eval[-1]
print(emin)

-7.249019570823107


TO BE DONE